# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather = pd.read_csv("output_data/cities.csv")
weather.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,thinadhoo,100,MV,1578358668,75,0.53,72.93,83.66,16.22
1,georgetown,40,GY,1578358109,83,6.80,-58.16,78.80,4.70
2,rawlins,40,US,1578358671,62,41.79,-107.24,25.00,12.75
3,lagoa,0,PT,1578358233,87,37.14,-8.45,52.00,4.70
4,gudari,0,IN,1578358674,87,19.34,83.79,55.18,2.37


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = weather[["Lat", "Lng"]].astype(float)

In [4]:
humidity = weather["Humidity"]
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=1)

fig = gmaps.figure(zoom_level=2, center=[0,0])
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# ideal weather: temperature between 70-80, wind < 10mph, 0 cloudiness
ideal_weather = weather.loc[(weather["Max Temp"] > 70) & (weather["Max Temp"] < 80)
                           & (weather["Wind Speed"] < 10) & (weather["Cloudiness"] == 0)]
ideal_weather

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
39,jamestown,0,AU,1578358219,41,-33.21,138.60,74.52,8.01
49,guerrero negro,0,MX,1578358693,30,27.97,-114.04,71.73,3.36
270,whakatane,0,NZ,1578358705,41,-37.96,176.99,72.00,5.01
410,kralendijk,0,BQ,1578358713,78,12.15,-68.27,78.80,4.70
462,sur,0,OM,1578358240,84,22.57,59.53,70.23,9.15


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
ideal_weather["Hotel Name"] = ""
ideal_weather

C:\Users\jbarn\.conda\envs\pydata\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
39,jamestown,0,AU,1578358219,41,-33.21,138.60,74.52,8.01,
49,guerrero negro,0,MX,1578358693,30,27.97,-114.04,71.73,3.36,
270,whakatane,0,NZ,1578358705,41,-37.96,176.99,72.00,5.01,
410,kralendijk,0,BQ,1578358713,78,12.15,-68.27,78.80,4.70,
462,sur,0,OM,1578358240,84,22.57,59.53,70.23,9.15,


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "radius": 5000,
    "keyword": "hotel",
    "key": g_key
}

for index, row in ideal_weather.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city = row["City"]
    params["location"] = f"{lat}, {lng}"
    response = requests.get(base_url, params=params).json()
    # create exception if no hotel is found
    try:
        results = response["results"][0]["name"]
        ideal_weather.loc[index, "Hotel Name"] = results
    except (IndexError):
        ideal_weather.loc[index, "Hotel Name"] = "N/A"
        print(f"No hotel found in {city}. Skipping...")
    
ideal_weather

C:\Users\jbarn\.conda\envs\pydata\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
39,jamestown,0,AU,1578358219,41,-33.21,138.60,74.52,8.01,Jamestown Hotel
49,guerrero negro,0,MX,1578358693,30,27.97,-114.04,71.73,3.36,"Casa Laguna, Bed & Breakfast"
270,whakatane,0,NZ,1578358705,41,-37.96,176.99,72.00,5.01,White Island Rendezvous Motel
410,kralendijk,0,BQ,1578358713,78,12.15,-68.27,78.80,4.70,Courtyard by Marriott Bonaire Dive Resort
462,sur,0,OM,1578358240,84,22.57,59.53,70.23,9.15,Sur Grand Hotel


In [8]:
# remove rows containing cities without hotels, rename to hotel_df
hotel_df = ideal_weather[(ideal_weather != "N/A").all(axis=1)]
    
hotel_df

C:\Users\jbarn\.conda\envs\pydata\lib\site-packages\pandas\core\ops\__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
39,jamestown,0,AU,1578358219,41,-33.21,138.60,74.52,8.01,Jamestown Hotel
49,guerrero negro,0,MX,1578358693,30,27.97,-114.04,71.73,3.36,"Casa Laguna, Bed & Breakfast"
270,whakatane,0,NZ,1578358705,41,-37.96,176.99,72.00,5.01,White Island Rendezvous Motel
410,kralendijk,0,BQ,1578358713,78,12.15,-68.27,78.80,4.70,Courtyard by Marriott Bonaire Dive Resort
462,sur,0,OM,1578358240,84,22.57,59.53,70.23,9.15,Sur Grand Hotel


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))